<a href="https://colab.research.google.com/github/stevenbowler/EmployeeSurvey/blob/main/EmployeeSurvey3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Cell 1 - Install everything
!apt-get update -qq
!apt-get install -y tesseract-ocr poppler-utils -qq
!pip install -q pdf2image huggingface_hub openai pillow pymupdf opencv-python pandas matplotlib

In [ ]:
# Cell 2 - import os, glob, json, base64, cv2, numpy as np
from io import BytesIO
from pathlib import Path
from collections import defaultdict
from pdf2image import convert_from_path
from openai import OpenAI
from huggingface_hub import snapshot_download
import matplotlib.pyplot as plt
import pandas as pd
from google.colab import userdata

# ---- your secrets -------------------------------------------------
HF_TOKEN   = userdata.get('HF_TOKEN')      # HF token for private repo
REPO_ID    = userdata.get('REPO_ID')       # e.g. "yourname/survey-pdfs"
XAI_API_KEY= userdata.get('XAI_API_KEY')   # xAI API key
PDF_DIR    = userdata.get('PDF_FOLDER_PATH') or "/content/pdfs"
# ------------------------------------------------------------------

In [ ]:
# Cell 3 Download PDFs
os.makedirs(PDF_DIR, exist_ok=True)
pdf_dir = snapshot_download(
    repo_id=REPO_ID,
    token=HF_TOKEN,
    local_dir=PDF_DIR,
    repo_type="dataset",
    allow_patterns=["*.pdf"],
    ignore_patterns=["*.json","*.md","*config*"]
)
pdf_paths = sorted(glob.glob(f"{pdf_dir}/**/*.pdf", recursive=True))
print(f"Found {len(pdf_paths)} PDF files")

In [ ]:
# Cell 4 - ROI definition (tuned to images)
NUM_QUESTIONS = 23          # Q1-20 on page 1, Q21-23 on page 2

# (x, y, w, h) for the four columns at DPI=300
COL_A = ( 570, 0, 35, 35)   # Strongly Agree
COL_B = ( 730, 0, 35, 35)   # Agree
COL_C = ( 890, 0, 35, 35)   # Disagree
COL_D = (1050, 0, 35, 35)   # Strongly Disagree

# y-offset for each question (measured from the top of the page)
Y_OFFSETS_PAGE1 = [
    205, 250, 295, 340, 385,          # Q1-Q5
    430, 475, 520, 565, 610,          # Q6-Q10
    655, 700, 745, 790, 835,          # Q11-Q15
    880, 925, 970,1015,1060           # Q16-Q20
]

Y_OFFSETS_PAGE2 = [205, 250, 295]      # Q21-Q23 (same spacing)

# Build a dict: page_index → question → list of 4 ROIs
CHECKBOX_ROIS = {}
# page 0 = first page
for q, y in enumerate(Y_OFFSETS_PAGE1, 1):
    CHECKBOX_ROIS.setdefault(0, {})[q] = [
        (COL_A[0], y+COL_A[1], COL_A[2], COL_A[3]),
        (COL_B[0], y+COL_B[1], COL_B[2], COL_B[3]),
        (COL_C[0], y+COL_C[1], COL_C[2], COL_C[3]),
        (COL_D[0], y+COL_D[1], COL_D[2], COL_D[3]),
    ]
# page 1 = second page
for q, y in enumerate(Y_OFFSETS_PAGE2, 21):
    CHECKBOX_ROIS.setdefault(1, {})[q] = [
        (COL_A[0], y+COL_A[1], COL_A[2], COL_A[3]),
        (COL_B[0], y+COL_B[1], COL_B[2], COL_B[3]),
        (COL_C[0], y+COL_C[1], COL_C[2], COL_C[3]),
        (COL_D[0], y+COL_D[1], COL_D[2], COL_D[3]),
    ]

In [ ]:
# Cell 5  - Image preprocessing + checkbox detection
def preprocess_image(img_cv):
    """Turn faint grey fills into solid black while keeping empty boxes white."""
    gray = cv2.cvtColor(img_cv, cv2.COLOR_BGR2GRAY)

    # 1. Adaptive contrast (CLAHE) – brings out faint marks
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    enhanced = clahe.apply(gray)

    # 2. Morphological closing to fill tiny holes inside a mark
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3))
    closed = cv2.morphologyEx(enhanced, cv2.MORPH_CLOSE, kernel)

    # 3. Adaptive threshold – works on uneven lighting
    thresh = cv2.adaptiveThreshold(
        closed, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY_INV, 31, 12)

    return thresh

def is_filled(roi, thresh_img, fill_ratio=0.45):
    """Return True if at least *fill_ratio* of the ROI is black."""
    x,y,w,h = roi
    crop = thresh_img[y:y+h, x:x+w]
    if crop.size == 0:
        return False
    black = np.sum(crop == 0)
    return black / (w*h) >= fill_ratio

def extract_answers(pdf_path):
    """Return list of 23 letters ['A','B',...] or 'X'."""
    try:
        pages = convert_from_path(pdf_path, dpi=300)
    except Exception as e:
        print(f"PDF conversion error {pdf_path}: {e}")
        return None

    answers = ['X'] * NUM_QUESTIONS
    for page_idx, pil_img in enumerate(pages):
        img_cv = cv2.cvtColor(np.array(pil_img), cv2.COLOR_RGB2BGR)
        thresh = preprocess_image(img_cv)

        # which questions live on this page?
        q_start = 1 if page_idx == 0 else 21
        q_end   = 20 if page_idx == 0 else 23

        for q in range(q_start, q_end+1):
            rois = CHECKBOX_ROIS.get(page_idx, {}).get(q, None)
            if not rois:
                continue
            for col_idx, roi in enumerate(rois, 1):   # 1=A,2=B,3=C,4=D
                if is_filled(roi, thresh):
                    answers[q-1] = chr(64 + col_idx)   # A=65 …
                    break
    return answers

In [ ]:
# Cell 6 - Process all PDFs
all_answers = []
for i, p in enumerate(pdf_paths, 1):
    print(f"[{i}/{len(pdf_paths)}] {Path(p).name}")
    ans = extract_answers(p)
    if ans and len(ans) == NUM_QUESTIONS:
        all_answers.append(ans)
    else:
        print("   → failed / incomplete")

print(f"\nSuccessfully parsed {len(all_answers)} / {len(pdf_paths)} PDFs")

In [ ]:
# Cell 7 - Aggregate percentages
stats = defaultdict(lambda: defaultdict(int))
for row in all_answers:
    for q, choice in enumerate(row, 1):
        if choice != 'X':
            stats[q][choice] += 1

percentages = {}
total_resp = len(all_answers)
for q, cnt in stats.items():
    tot = sum(cnt.values())
    if tot:
        per = {c: round(v/tot*100, 2) for c,v in cnt.items()}
        percentages[q] = dict(sorted(per.items(), key=lambda x:-x[1]))

print(f"Total responses: {total_resp}")
for q, p in percentages.items():
    print(f"Q{q:02d}: {p}")

In [ ]:
# Cell 8 - One-call Grok 4 summary
client = OpenAI(api_key=XAI_API_KEY, base_url="https://api.x.ai/v1")
MODEL = "grok-4"

summary_prompt = f"""You are a data-analyst.
Here are the aggregated results from {total_resp} employees (23 questions, A=Strongly Agree … D=Strongly Disagree).

Percentages (most popular first):
{json.dumps(percentages, indent=2)}

Write a concise professional report with:
1. Overall insights (strongest/weakest areas)
2. Markdown table of the top answer % per question
3. Any noticeable patterns or anomalies
4. Short visual description (as if you had bar-charts)
5. 2-3 actionable recommendations

Use markdown, keep it under 800 words."""

resp = client.chat.completions.create(
    model=MODEL,
    messages=[{"role":"user","content":summary_prompt}],
    max_tokens=2000,
    temperature=0.3
)
print("\n### Grok-4 Summary Report")
print(resp.choices[0].message.content)

In [ ]:
# Cell 9 - Charts and CSV Report
# ----- bar charts -----
rows = (NUM_QUESTIONS // 5) + (1 if NUM_QUESTIONS % 5 else 0)
fig, axs = plt.subplots(rows, 5, figsize=(22, 4*rows))
axs = axs.ravel()

for idx, (q, per) in enumerate(percentages.items()):
    choices = list(per.keys())
    vals    = list(per.values())
    axs[idx].bar(choices, vals, color=['#2ca02c','#1f77b4','#ff7f0e','#d62728'])
    axs[idx].set_title(f'Q{q}')
    axs[idx].set_ylim(0,100)

for ax in axs[len(percentages):]:
    ax.axis('off')
plt.tight_layout()
plt.show()

# ----- CSV -----
df = pd.DataFrame(percentages).T.fillna(0)
csv_path = f"{PDF_DIR}/survey_percentages.csv"
df.to_csv(csv_path)
print(f"CSV saved → {csv_path}")

In [ ]:
#optional debug - visualize the ROIs on one page
def debug_roi(pdf_path, page_idx=0):
    pages = convert_from_path(pdf_path, dpi=300)
    img = cv2.cvtColor(np.array(pages[page_idx]), cv2.COLOR_RGB2BGR)
    for q, rois in CHECKBOX_ROIS.get(page_idx, {}).items():
        for col, (x,y,w,h) in enumerate(rois, 1):
            cv2.rectangle(img, (x,y), (x+w,y+h), (0,255,0), 2)
            cv2.putText(img, f"Q{q}{chr(64+col)}", (x,y-5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 1)
    from google.colab.patches import cv2_imshow
    cv2_imshow(img)

debug_roi(pdf_paths[0])   # change index to any PDF you want to inspect